In [ ]:
# !pip install catboost

In [6]:
import pandas as pd
import catboost
from catboost import CatBoostClassifier
from catboost import Pool, cv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [7]:
df=pd.read_csv("/content/train.csv")

In [8]:
df.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [9]:
df.columns

Index(['id', 'CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender',
       'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [10]:
# Assuming df is your DataFrame
X = df.drop('Exited', axis=1)  # Replace 'target_column' with the actual name of your target column
y = df['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

categorical_features_indices = np.where(X.dtypes != np.float)[0]

train_data = Pool(data=X_train, label=y_train, cat_features=categorical_features_indices)
test_data = Pool(data=X_test, label=y_test, cat_features=categorical_features_indices)


In [11]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
import numpy as np

In [14]:
# categorical_features_indices = np.where(X.dtypes != np.float)[0]


<ipython-input-14-65eb6c95badc>:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  categorical_features_indices = np.where(X.dtypes != np.float)[0]


In [15]:
# train_data = Pool(data=X_train, label=y_train, cat_features=categorical_features_indices)
# test_data = Pool(data=X_test, label=y_test, cat_features=categorical_features_indices)


In [16]:
model = CatBoostClassifier(iterations=500,  # Adjust the number of iterations as needed
                           depth=6,  # Adjust the depth of the trees
                           learning_rate=0.1,  # Adjust the learning rate
                           loss_function='Logloss',  # Use Logloss for binary classification
                           cat_features=categorical_features_indices,  # Specify categorical features
                           random_seed=42  # Set a random seed for reproducibility
                           )

model.fit(train_data, eval_set=test_data, early_stopping_rounds=50, verbose=100)


0:	learn: 0.6016662	test: 0.6016674	best: 0.6016674 (0)	total: 472ms	remaining: 3m 55s
100:	learn: 0.3093568	test: 0.3077227	best: 0.3077227 (100)	total: 33.9s	remaining: 2m 14s
200:	learn: 0.3037676	test: 0.3052751	best: 0.3052751 (200)	total: 1m 7s	remaining: 1m 40s
300:	learn: 0.3004812	test: 0.3046713	best: 0.3046453 (285)	total: 1m 40s	remaining: 1m 6s
400:	learn: 0.2975760	test: 0.3043972	best: 0.3043890 (384)	total: 2m 14s	remaining: 33.3s
499:	learn: 0.2949642	test: 0.3042315	best: 0.3042315 (499)	total: 2m 47s	remaining: 0us

bestTest = 0.3042314634
bestIteration = 499



In [17]:
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.8713000272669433
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.95      0.92     26052
           1       0.76      0.57      0.65      6955

    accuracy                           0.87     33007
   macro avg       0.83      0.76      0.79     33007
weighted avg       0.86      0.87      0.86     33007

Confusion Matrix:
 [[24798  1254]
 [ 2994  3961]]


In [24]:
model = CatBoostClassifier(iterations=448,  # Adjust the number of iterations as needed
                           depth=6,  # Adjust the depth of the trees
                           learning_rate=0.1,  # Adjust the learning rate
                           loss_function='Logloss',  # Use Logloss for binary classification
                           cat_features=categorical_features_indices,  # Specify categorical features
                           random_seed=42,  # Set a random seed for reproducibility
                           eval_metric='AUC'  # Specify the evaluation metric as AUC
                           )

model.fit(train_data, eval_set=test_data, early_stopping_rounds=50, verbose=100)


0:	test: 0.8737338	best: 0.8737338 (0)	total: 709ms	remaining: 5m 16s
100:	test: 0.8979804	best: 0.8979804 (100)	total: 34.1s	remaining: 1m 57s
200:	test: 0.8995876	best: 0.8995876 (200)	total: 1m 8s	remaining: 1m 23s
300:	test: 0.9000234	best: 0.9000351 (295)	total: 1m 42s	remaining: 49.9s
400:	test: 0.9002009	best: 0.9002097 (395)	total: 2m 16s	remaining: 16s
447:	test: 0.9003033	best: 0.9003033 (447)	total: 2m 31s	remaining: 0us

bestTest = 0.9003032618
bestIteration = 447



In [19]:
# Get predicted probabilities for the positive class
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Compute AUC-ROC
from sklearn.metrics import roc_auc_score
auc_roc = roc_auc_score(y_test, y_pred_proba)
print("AUC-ROC:", auc_roc)


AUC-ROC: 0.9003032617505684


In [27]:
import pandas as pd

# Load the test dataset
test_data = pd.read_csv('/content/test.csv')

# Assuming 'id' is the name of the ID column in your test set
# Use the test data for making predictions
test_predictions_proba = model.predict(test_data, prediction_type='Probability')[:, 1]

# Create a DataFrame with predictions
submission_df = pd.DataFrame({'id': test_data['id'], 'Exited': test_predictions_proba})

# Save the DataFrame to a CSV file
submission_df.to_csv('submission1.csv', index=False)



In [32]:
import pandas as pd

# Assuming your DataFrame is named 'df' and the target column is 'Exited'
target_counts = df['Exited'].value_counts()

print(target_counts)


0    130113
1     34921
Name: Exited, dtype: int64


In [33]:
# Check the class distribution (optional)
print(y_train.value_counts())

# Initialize CatBoostClassifier with automatic class weights
model = CatBoostClassifier(eval_metric='AUC', auto_class_weights='Balanced', random_seed=42, verbose=200)

# Fit the model to the training data
model.fit(X_train, y_train)


0    104061
1     27966
Name: Exited, dtype: int64


CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=2]="Yobachukwu": Cannot convert 'b'Yobachukwu'' to float

In [34]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report


In [35]:
# Assuming 'Exited' is your target column
X = df.drop('Exited', axis=1)
y = df['Exited']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [36]:
print(y_train.value_counts())


0    104061
1     27966
Name: Exited, dtype: int64


In [38]:
categorical_features = X.select_dtypes(include=['object']).columns
print("Categorical Features:", categorical_features)


Categorical Features: Index(['Surname', 'Geography', 'Gender'], dtype='object')


In [40]:
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report


In [41]:
# Assuming 'Exited' is your target column
X = df.drop('Exited', axis=1)
y = df['Exited']

# Identify categorical features
categorical_features = X.select_dtypes(include=['object']).columns

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [43]:
# Check the class distribution (optional)
print(y_train.value_counts())

# Identify categorical features
categorical_features = list(X.select_dtypes(include=['object']).columns)

# Initialize CatBoostClassifier with automatic class weights and categorical features
model = CatBoostClassifier(eval_metric='AUC', auto_class_weights='Balanced', cat_features=categorical_features, random_seed=42, verbose=200)

# Fit the model to the training data
model.fit(X_train, y_train)



0    104061
1     27966
Name: Exited, dtype: int64
Learning rate set to 0.082881
0:	total: 145ms	remaining: 2m 25s
200:	total: 26.5s	remaining: 1m 45s
400:	total: 53.6s	remaining: 1m 20s
600:	total: 1m 22s	remaining: 54.6s
800:	total: 1m 50s	remaining: 27.6s
999:	total: 2m 18s	remaining: 0us


In [44]:
# Predict on the test set
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Calculate AUC-ROC score
auc_roc_score = roc_auc_score(y_test, y_pred_proba)
print(f'AUC-ROC Score: {auc_roc_score:.4f}')

# Print classification report
y_pred = model.predict(X_test)
print('\nClassification Report:')
print(classification_report(y_test, y_pred))


AUC-ROC Score: 0.8960

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.82      0.88     26052
           1       0.55      0.80      0.65      6955

    accuracy                           0.82     33007
   macro avg       0.74      0.81      0.76     33007
weighted avg       0.86      0.82      0.83     33007



In [61]:
df=pd.read_csv("/content/train.csv")

In [62]:
# Assuming df is your DataFrame
X = df.drop('Exited', axis=1)  # Replace 'target_column' with the actual name of your target column
y = df['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

categorical_features_indices = np.where(X.dtypes != np.float)[0]

train_data = Pool(data=X_train, label=y_train, cat_features=categorical_features_indices)
test_data = Pool(data=X_test, label=y_test, cat_features=categorical_features_indices)

<ipython-input-62-b5e452878d4e>:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  categorical_features_indices = np.where(X.dtypes != np.float)[0]


In [63]:
model = CatBoostClassifier(iterations=500,  # Adjust the number of iterations as needed
                           depth=6,  # Adjust the depth of the trees
                           learning_rate=0.062881,  # Adjust the learning rate
                           loss_function='Logloss',  # Use Logloss for binary classification
                           cat_features=categorical_features_indices,  # Specify categorical features
                           random_seed=42,  # Set a random seed for reproducibility
                           eval_metric='AUC'  # Specify the evaluation metric as AUC
                           )

model.fit(train_data, eval_set=test_data, early_stopping_rounds=50, verbose=100)

0:	test: 0.8737338	best: 0.8737338 (0)	total: 372ms	remaining: 3m 5s
100:	test: 0.8966095	best: 0.8966095 (100)	total: 36s	remaining: 2m 22s
200:	test: 0.8988085	best: 0.8988113 (199)	total: 1m 8s	remaining: 1m 42s
300:	test: 0.8996177	best: 0.8996177 (300)	total: 1m 42s	remaining: 1m 7s
400:	test: 0.8999604	best: 0.8999604 (400)	total: 2m 16s	remaining: 33.7s
499:	test: 0.9001708	best: 0.9001753 (491)	total: 2m 50s	remaining: 0us

bestTest = 0.9001752757
bestIteration = 491

Shrink model to first 492 iterations.


In [64]:
# Get predicted probabilities for the positive class
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Compute AUC-ROC
from sklearn.metrics import roc_auc_score
auc_roc = roc_auc_score(y_test, y_pred_proba)
print("AUC-ROC:", auc_roc)

AUC-ROC: 0.9001752757273707


In [65]:
import pandas as pd

# Load the test dataset
test_data = pd.read_csv('/content/test.csv')

# Assuming 'id' is the name of the ID column in your test set
# Use the test data for making predictions
test_predictions_proba = model.predict(test_data, prediction_type='Probability')[:, 1]

# Create a DataFrame with predictions
submission_df = pd.DataFrame({'id': test_data['id'], 'Exited': test_predictions_proba})

# Save the DataFrame to a CSV file
submission_df.to_csv('submission2.csv', index=False)